In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir
from geo_tool import Point_Cloud

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [20]:
class_name = 'chair'
do_mmd = True
do_cov = False
sample_metrics = False
mmd_loss = 'emd'
boost_samples = 3
skip = ['test', 'train']
n_repeats = 1
report_gmm = True
single_con_comp = True  # Use point-samples from largest-connected-component of voxel/mesh.

In [21]:
top_syn_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/voxel_based/'

sample_files = []

gmm_tags = ['ae_chair_phuoc_data_64_voxels_64_bneck', 
            'ae_chair_sn_vox_data_32_voxels_64_bneck']

gmm_tags = ['ae_chair_sn_vox_data_32_voxels_64_bneck']

if single_con_comp:
    sample_tag = 'samples_with_lcc.npz'
else:
    sample_tag = 'samples.npz'

if report_gmm:
    for tag in gmm_tags:
        sample_files.append(osp.join(top_syn_dir, 'gmm', tag, sample_tag))

In [22]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()

In [23]:
for sample_file in sample_files:
    print sample_file
    mmd_scores = []
    cov_scores = []
    for _ in range(n_repeats):        
        evaluator.prepare_sample_data(sample_file, boost_samples)

        if 'phuoc' in sample_file:
            # Phuoc rotated -90 compared to GT.
            for k, v in evaluator.sample_data.iteritems():
                temp = np.zeros_like(v)
                for i in xrange(len(v)):
                    temp[i] = Point_Cloud(v[i]).rotate_z_axis_by_degrees(-90).points
                evaluator.sample_data[k] = temp

        if do_mmd:
            score = evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_metrics, skip=skip)
            mmd_scores.append(score)

        if do_cov:
            score = evaluator.compute_coverage(loss=mmd_loss, sample_estimator=sample_metrics, skip=skip)
            cov_scores.append(score)
    
    for score_type in [mmd_scores, cov_scores]:
        if len(score_type) > 0:
            total = 0.0
            for s in score_type:
                total += np.mean(s['val'])
            print total / len(score_type)

/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/voxel_based/gmm/ae_chair_sn_vox_data_32_voxels_64_bneck/samples_with_lcc.npz
val 152.752 34.8252
152.752090454


In [ ]:
# #For histogram comparissons (incomplete code)
# from general_tools.in_out.basics import create_dir
# out_d = osp.join(top_evaluation_dir, 'dist_histograms', class_name)
# create_dir(out_d)
# out_f = osp.join(out_d, 'mmd_%s_dists_32_vox_val_split' % (mmd_loss,) )
# print out_f
# np.savez(out_f, mmd_scores[0]['val'])

# l_h = np.load('l_emd_hist_test.npz')
# l_h = l_h[l_h.keys()[0]]
# r_h = np.array(all_d['test'])
# bins = np.linspace(-0.000001, np.max([r_h, l_h]), 100)
# plt.hist(r_h, bins, alpha=0.5, label='raw')
# plt.hist(l_h, bins, alpha=0.5, label='l_emd')
# plt.grid()
# plt.legend(loc='upper right')